source activate ENVNAME
pip install ipykernel
python -m ipykernel install --user --name ENVNAME --display-name "Python (whatever you want to call it)"

pip install --upgrade google-api-python-client
pip install -t lib --upgrade google-api-python-client && pip freeze > requirements.txt
pip install -t lib --upgrade google-cloud-bigquery && pip freeze > requirements.txt
pip install -t lib bigquery-python && pip freeze > requirements.txt
pip install -t lib --upgrade google-cloud

$ virtualenv venv
$ source venv/bin/activate
(venv) $ venv/bin/pip install --upgrade google-cloud
(venv) $ venv/bin/python -c 'import google.cloud.storage'
(venv) $ deactivate
$ rm -fr venv/

venv/bin/pip install --upgrade google-api-python-client
venv/bin/pip install --upgrade webapp2
venv/bin/pip install --upgrade webob
venv/bin/pip install --upgrade Flask

In [ ]:
from googleapiclient.discovery import build
#from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from urllib2 import HTTPError
#from slackclient import SlackClient
import json

In [ ]:
def get_service(api_name, api_version, scope, key_file_location):
    credentials = ServiceAccountCredentials.from_json_keyfile_name(key_file_location, scope)
    http = credentials.authorize(httplib2.Http())
    service = build(api_name, api_version, http=http)

    return service


def get_first_profile_id(service):
    accounts = service.management().accounts().list().execute()

    if accounts.get('items'):
        account = accounts.get('items')[0].get('id')
        properties = service.management().webproperties().list(accountId=account).execute()

        if properties.get('items'):
            property = properties.get('items')[0].get('id')
            profiles = service.management().profiles().list(accountId=account, webPropertyId=property).execute()

        if profiles.get('items'):
            return profiles.get('items')[0].get('id')

    return None


def get_results(service, profile_id, metrics, dimensions, sort):
    try:
        return service.data().realtime().get(
            ids='ga:' + profile_id,
            metrics=metrics,
            dimensions=dimensions,
            sort=sort
        ).execute()

    except TypeError, error:
        # Handle errors in constructing a query.
        print ('There was an error in constructing your query : %s' % error)

    except HTTPError, error:
        # Handle API errors.
        print ('Arg, there was an API error : %s : %s' %
               (error.resp.status, error._get_reason()))


def get_detailed_totals(results):
    output = []

    totals = results.get('totalsForAllResults')
    for metric_name, metric_total in totals.iteritems():
        output.append(metric_total)

    if results.get('rows', []):
        detailed = ''
        for row in results.get('rows')[0:10]:
            if row[0] == '(not set)':
                detail = ''
            else:
                detail = row[0]
            detailed += '*' + row[2] + '* ' + row[1] + detail + '\n'

        output.append(detailed)

    return output

In [ ]:
def main():
    response = ''
    scope = ['https://www.googleapis.com/auth/analytics.readonly']
    key_file_location = '/Users/jose/Documents/GitHub/google_cloud_playground/real_time_dashboard/bankinterconsumerfinance_ga-api-6c584fc0cf81.json'

    service = get_service('analytics', 'v3', scope, key_file_location)
    profile = get_first_profile_id(service)
    print(profile)

    metrics = 'rt:activeUsers'
    dimensions = 'rt:minutesAgo,rt:source'
    sort = '-rt:activeUsers'
    
    results = get_results(service, profile, metrics=metrics, dimensions=dimensions, sort=sort)
    print(results)

    detailed = get_detailed_totals(results)
    total_active = detailed[0]
    details = detailed[1]
    print(total_active)
    return total_active

In [ ]:
main()

In [ ]:
import datetime
def query_rt():
    response = ''
    scope = ['https://www.googleapis.com/auth/analytics.readonly']
    key_file_location = '/Users/jose/Documents/GitHub/google_cloud_playground/real_time_dashboard/bankinterconsumerfinance_ga-api-6c584fc0cf81.json'

    service = get_service('analytics', 'v3', scope, key_file_location)
    profile = get_first_profile_id(service)

    metrics = 'rt:activeUsers'
    dimensions = 'rt:minutesAgo'
    sort = '-rt:activeUsers'
    
    results = get_results(service, profile, metrics=metrics, dimensions=dimensions, sort=sort)
    #print(results)
    totals = results.get('totalsForAllResults')
    #for metric_name, metric_total in totals.iteritems():
    #    print(metric_total)
    #print(totals.keys()[0])
    #print(totals['rt:activeUsers'])
    ds = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    #print(ds)
    totals['ds'] = ds
    totals['y'] = totals.pop('rt:activeUsers')
    return totals


In [ ]:
query_rt()

In [ ]:
def stream_data(dataset_name, table_name, json_data, time_stamp = time.time()):
    bigquery_client = bigquery.Client()
    dataset = bigquery_client.dataset(dataset_name)
    table = dataset.table(table_name)
    data = json_data

    data['ts'] = time_stamp

    # Reload the table to get the schema.
    table.reload()

    ## get the names of schema
    schema = table.schema
    schema_names = [o.name for o in schema]

    logging.debug('BQ Schema: {}'.format(schema_names))

    # from schema names get list of tuples of the values
    rows = [(data[x] for x in schema_names)]

    # https://googlecloudplatform.github.io/google-cloud-python/stable/bigquery-table.html#google.cloud.bigquery.table.Table.insert_data
    errors = table.insert_data(rows, row_ids = str(uuid.uuid4()))

    if not errors:
    	logging.debug('Loaded 1 row into {}:{}'.format(dataset_name, table_name))
    else:
        logging.error(errors)

In [3]:
from google.cloud import bigquery

ImportError: No module named google.cloud

In [4]:
bigquery_client = bigquery.Client.from_service_account_json(
        './bq-ga-api-6175982920c6.json')

NameError: name 'bigquery' is not defined

In [ ]:
def list_datasets(project=None):
    """Lists all datasets in a given project.

    If no project is specified, then the currently active project is used.
    """
    #bigquery_client = bigquery.Client(project=project)
    bigquery_client = bigquery.Client.from_service_account_json(
        './bq-ga-api-6175982920c6.json',project=project)

    for dataset in bigquery_client.list_datasets():
        print(dataset.name)

In [ ]:
billing_projectId = "eam-ga-api"

In [ ]:
list_datasets(billing_projectId)

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/jose/Documents/GitHub/google_cloud_playground/real_time_dashboard/bq-ga-api-6175982920c6.json"

In [ ]:
bigquery_client = bigquery.Client()
bigquery_client = bigquery_client.from_service_account_json('./bq-ga-api-6175982920c6.json')


In [ ]:
bigquery_client

In [ ]:
client.project

gcloud auth application-default login
Utilizamos el SDK
https://github.com/GoogleCloudPlatform/python-docs-samples/tree/e5c0ff2865c1eac87fb7279b5d381c736d7a52e2/bigquery/cloud-client

In [5]:
def implicit():
    from google.cloud import bigquery

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    bigquery_client = bigquery.Client()

    # Make an authenticated API request
    datasets = list(bigquery_client.list_datasets())
    print(datasets)

In [6]:
implicit()

ImportError: No module named google.cloud

In [ ]:
def explicit():
    from google.cloud import bigquery

    # Explicitly use service account credentials by specifying the private key
    # file. All clients in google-cloud-python have this helper, see
    # https://google-cloud-python.readthedocs.io/en/latest/core/modules.html
    #   #google.cloud.client.Client.from_service_account_json
    bigquery_client = bigquery.Client.from_service_account_json(
        '/Users/jose/Documents/GitHub/google_cloud_playground/real_time_dashboard/bq-ga-api-6175982920c6.json')

    # Make an authenticated API request
    buckets = list(bigquery_client.list_datasets())
    print(buckets)

In [ ]:
explicit()

In [ ]:
def list_datasets(project=None):
    """Lists all datasets in a given project.
    If no project is specified, then the currently active project is used.
    """
    bigquery_client = bigquery.Client(project=project)

    for dataset in bigquery_client.list_datasets():
        print(dataset.dataset_id)

In [ ]:
list_datasets()

In [ ]:
def run_quickstart():
    # [START bigquery_quickstart]
    # Imports the Google Cloud client library
    from google.cloud import bigquery

    # Instantiates a client
    bigquery_client = bigquery.Client()

    # The name for the new dataset
    dataset_id = 'poc'

    # Prepares a reference to the new dataset
    dataset_ref = bigquery_client.dataset(dataset_id)
    dataset = bigquery.Dataset(dataset_ref)

    # Creates the new dataset
    dataset = bigquery_client.create_dataset(dataset)

    print('Dataset {} created.'.format(dataset.dataset_id))
    # [END bigquery_quickstart]

In [ ]:
run_quickstart()

In [ ]:
def create_table(dataset_id, table_id, project=None):
    """Creates a simple table in the given dataset.
    If no project is specified, then the currently active project is used.
    """
    bigquery_client = bigquery.Client(project=project)
    dataset_ref = bigquery_client.dataset(dataset_id)

    table_ref = dataset_ref.table(table_id)
    table = bigquery.Table(table_ref)

    # Set the table schema
    table.schema = (
        bigquery.SchemaField('ds', 'TIMESTAMP'),
        bigquery.SchemaField('y', 'INTEGER'),
    )

    table = bigquery_client.create_table(table)

    print('Created table {} in dataset {}.'.format(table_id, dataset_id))

In [ ]:
create_table('poc','streaming')

In [ ]:
import time
def stream_data(dataset_id, table_id, json_data, time_stamp = time.time()):
    bigquery_client = bigquery.Client()
    dataset_ref = bigquery_client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    print(json_data)
    data = json_data
    #data = json.loads(json_data)
    
    #data['ts'] = time_stamp
    print(time_stamp)
    print(data)

    # Get the table from the API so that the schema is available.
    table = bigquery_client.get_table(table_ref)

    rows = [data]
    errors = bigquery_client.create_rows(table, rows)

    if not errors:
        print('Loaded 1 row into {}:{}'.format(dataset_id, table_id))
    else:
        print('Errors:')
        pprint(errors)

In [ ]:
import json
from pprint import pprint

In [ ]:
DATASET_ID = 'poc'
TABLE_ID = 'streaming'
RT_DATA = query_rt()
print(RT_DATA)
def test_stream_data():
    stream_data(
        DATASET_ID,
        TABLE_ID,
        RT_DATA)

In [ ]:
test_stream_data()

In [ ]:
dataset_name = 'streaming'
table_name = 'foo'
dataset = bigquery_client.dataset(dataset_name)
table = dataset.table(table_name)
#data = json_data
data = '["Gandalf", 2000, 1.2]'
print(table)

In [ ]:
table.schema